In [1]:
import cv2
import numpy as np
from numpy.core.numeric import True_
from numpy.lib.arraysetops import unique
import pandas as pd
# import glob
import os
from pandas import json_normalize
from os import getcwd, path
from yaml import SafeLoader, load
import datetime
import matplotlib.pyplot as plt
import yaml
import shutil
import json

from video_utils_virat import VideoUtils
from video_utils_virat import YoloUtils

%matplotlib inline

## Dataset Selection

In [2]:

# setup
dataset_dir_path = '../datasets/VIRAT/'
image_ext = '.jpg'
video_max_frames = 2000

#video
video_ext = '.mp4'
video_name = 'VIRAT_S_040102_04_000596_000689'
video_name_orig = video_name + video_ext
video_dest_path = './processed/' +  video_name + '/'
video_src_path = dataset_dir_path + 'Videos/Ground/'

using_yml = True

# annotations
saved_csv = video_dest_path + 'df_bbox.csv'


yml_video_name = 'gt_'
yolo_video_name = 'yolo_'

# annotations_path = dataset_dir_path + 'viratannotations/train/' + video_name +'/'
annotations_path = dataset_dir_path + 'viratannotations/validate/' + video_name +'/'
ann_activities_file = annotations_path + video_name + '.activities.yml'
ann_geom_file = annotations_path + video_name + '.geom.yml'
ann_regions_file = annotations_path + video_name + '.regions.yml'
ann_types_file = annotations_path + video_name + '.types.yml'


gt_video_name = yml_video_name + video_name + '.avi'#video_ext
yolo_video_name = yolo_video_name + video_name + '.avi'#video_ext

In [3]:
# Create directory to store new video
if not os.path.exists(video_dest_path):
    os.makedirs(video_dest_path)

if not os.path.exists(saved_csv):
    with open(ann_types_file) as yaml_file:
        yaml_contents = load(yaml_file, Loader=SafeLoader)
    yaml_df = json_normalize(yaml_contents)
    yaml_df
    for col in yaml_df.columns:
        type_name = col.split('.')[-1]
        if not (type_name == 'id1'):
            yaml_df.loc[yaml_df[col] == 1, col] = type_name
    
    yaml_df = yaml_df[yaml_df['types.id1'].notna()].reset_index().dropna(axis=1, how='all')  
    type_df = yaml_df.ffill(axis=1).iloc[:,-1].to_frame(name='category')
    type_df.insert(0, "id", yaml_df['types.id1'])
    type_df


In [4]:
# using annotations:
print("Loading annotations...")
def add_category_type(row):
  id = row['object_id']
  val = type_df.loc[type_df['id'] == id, 'category'].iloc[0]
  return val


if os.path.exists(saved_csv):
  df_bbox = pd.read_csv(saved_csv)
else:
  with open(ann_geom_file) as yaml_file:
      yaml_contents = load(yaml_file, Loader=SafeLoader)
  yaml_df = json_normalize(yaml_contents)

  df_bbox = yaml_df[['geom.id1','geom.ts0','geom.ts1','geom.g0']].dropna().reset_index()
  df_bbox.rename(columns={'geom.id1': 'object_id', 'geom.ts0': 'frame_id','geom.ts1': 'time_sec', 'geom.g0': 'bbox'}, inplace=True)
  df_bbox['bbox'] = df_bbox['bbox'].str.split()
  df_tmp = pd.DataFrame(df_bbox['bbox'].to_list(), columns = ['bb_left', 'bb_top', 'bb_right', 'bb_bottom'])
  df_bbox = pd.concat([df_bbox, df_tmp], axis=1).drop(columns=['bbox'])

  df_bbox['category'] = df_bbox.apply(lambda row: add_category_type(row), axis=1) 
  df_bbox.drop(columns=['index'], axis=1, inplace=True)
  # df_bbox.set_index['index'] 
  df_bbox.to_csv(saved_csv, index = False)
    

df_bbox.head()

Loading annotations...


,object_id,frame_id,time_sec,bb_left,bb_top,bb_right,bb_bottom,category
0,5.0,1561.0,52.033333,349,1049,415,1078,Person
1,5.0,1562.0,52.066667,355,1022,421,1078,Person
2,5.0,1563.0,52.100000,361,1022,427,1078,Person
3,5.0,1564.0,52.133333,361,1022,427,1078,Person
4,5.0,1565.0,52.166667,361,1022,427,1078,Person


# Add velocities

In [5]:
# Add velocities for the video

vidcap = cv2.VideoCapture(video_src_path + video_name_orig)
fps = 30
scale = 1/15
if vidcap.isOpened():
    fps = vidcap.get(cv2.CAP_PROP_FPS)  
    print(fps)
    # scale = #TBD


def find_mid(a, b):
  c = (a.astype(float) + b.astype(float))/2
  return c

def find_delta(x):
  dx = x.diff()
  dx[0] = np.nan
  dx.fillna(method='backfill', inplace=True)
  return dx

def find_vel(vx,vy):

  vel = np.sqrt(vx * vx + vy * vy).round(2)
  return vel#, vx, vy

for id in df_bbox['object_id'].unique():
  mask = (df_bbox['object_id']==id)
  sub_df = df_bbox[mask]
  x  = find_mid(sub_df['bb_top'], sub_df['bb_bottom'])
  y  = find_mid(sub_df['bb_left'], sub_df['bb_right'])
  dx = find_delta(x)
  dy = find_delta(y)
  vx = (dx * fps * scale)
  vy = (dy * fps * scale)

  df_bbox.loc[mask, 'x']  = x
  df_bbox.loc[mask, 'y']  = y
  df_bbox.loc[mask, 'dx'] = dx
  df_bbox.loc[mask, 'dy'] = dy
  df_bbox.loc[mask, 'vx'] = (dx * fps * scale)
  df_bbox.loc[mask, 'vy'] = (dy * fps * scale)
  df_bbox.loc[mask, 'vel']  = np.sqrt(vx * vx + vy * vy).round(2)

df_bbox.head()

29.97002997002997


,object_id,frame_id,time_sec,bb_left,bb_top,bb_right,bb_bottom,category,x,y,dx,dy,vx,vy,vel
0,5.0,1561.0,52.033333,349,1049,415,1078,Person,1063.5,382.0,-13.5,6.0,-26.973027,11.988012,29.52
1,5.0,1562.0,52.066667,355,1022,421,1078,Person,1050.0,388.0,-13.5,6.0,-26.973027,11.988012,29.52
2,5.0,1563.0,52.100000,361,1022,427,1078,Person,1050.0,394.0,0.0,6.0,0.000000,11.988012,11.99
3,5.0,1564.0,52.133333,361,1022,427,1078,Person,1050.0,394.0,0.0,0.0,0.000000,0.000000,0.00
4,5.0,1565.0,52.166667,361,1022,427,1078,Person,1050.0,394.0,0.0,0.0,0.000000,0.000000,0.00


In [6]:
from video_utils_virat import YoloUtils
from video_utils_virat import VideoUtils

yUtils = YoloUtils()
vUtils = VideoUtils() 

yolo_weight_file = "../../yolo/darknet/yolov3.weights"
yolo_cfg_file = "../../yolo/darknet/cfg/yolov3.cfg"
yolo_names_file = "../../yolo/darknet/data/coco.names"
start_time = 0
video_duration=1#None

yolo_video_out, bbox_yolo = yUtils.YoloOnVideo(yolo_weight_file, yolo_cfg_file, yolo_names_file, video_dest_path, video_src_path + video_name_orig, yolo_video_name, start_time_sec=start_time, duration_sec=video_duration, save_images=False)
gt_video_out, bbox_gt = vUtils.AnnotateVideo(video_dest_path, video_src_path + video_name_orig, gt_video_name, df_bbox, start_time, video_duration)
# bbox_gt

../datasets/VIRAT/Videos/Ground/VIRAT_S_040102_04_000596_000689.mp4
opened video
Total frames in video: 2779.00 @ 29.97 frames/sec
Created frame id  0, 0.00 sec in video; completed:  0.0 %
Created frame id 25, 0.83 sec in video; completed:  86.2 %
Done: Created video: ./processed/VIRAT_S_040102_04_000596_000689/0-0_yolo_VIRAT_S_040102_04_000596_000689.avi
Total frames in video: 2779.00 @ 29.97 frames/sec
2779.0 0 29
Created frame id  0, 0.00 sec in video; completed:  0.0 %
Created frame id 25, 0.83 sec in video; completed:  86.2 %
Done: Created video: ./processed/VIRAT_S_040102_04_000596_000689/0-0_gt_VIRAT_S_040102_04_000596_000689.avi


In [7]:
from video_utils_virat import Bbox


print(len(bbox_gt))

missed = []
hit = []
for i in range(len(bbox_gt)):
# for i in range(1):
    frame_missed = 0
    frame_hit = 0
    for bb_gt in bbox_gt[i]:
        # for j in bbox_yolo:
            for bb_pred in bbox_yolo[i]:
                gt = Bbox(bb_gt[0],bb_gt[1],bb_gt[2],bb_gt[3])
                pred = Bbox(bb_pred[0],bb_pred[1],bb_pred[2],bb_pred[3])
                iou = gt.IOU(pred)

                if(iou > 0.0):
                    print(f'frame {i}, ground truth bb:{bb_gt}, yolo:{bb_pred}, iou:{iou:0.3}')
                    frame_hit += 1
                    break
                else:
                    frame_missed += 1

    missed.append(frame_missed)
    hit.append(frame_hit)
print(missed)
print(hit)

30
frame 0, ground truth bb:[232, 501, 445, 720], yolo:[252, 515, 440, 700], iou:0.747
frame 0, ground truth bb:[658, 637, 997, 933], yolo:[637, 638, 1016, 919], iou:0.854
frame 0, ground truth bb:[866, 581, 1170, 818], yolo:[637, 638, 1016, 919], iou:0.179
frame 0, ground truth bb:[1152, 493, 1464, 704], yolo:[865, 582, 1155, 817], iou:0.00366
frame 0, ground truth bb:[354, 346, 525, 485], yolo:[362, 361, 530, 476], iou:0.771
frame 0, ground truth bb:[488, 306, 680, 445], yolo:[362, 361, 530, 476], iou:0.0851
frame 0, ground truth bb:[624, 301, 821, 424], yolo:[755, 296, 910, 386], iou:0.175
frame 0, ground truth bb:[741, 285, 917, 392], yolo:[755, 296, 910, 386], iou:0.743
frame 0, ground truth bb:[834, 258, 1026, 376], yolo:[755, 296, 910, 386], iou:0.202
frame 1, ground truth bb:[232, 501, 445, 720], yolo:[252, 515, 441, 699], iou:0.747
frame 1, ground truth bb:[658, 637, 997, 933], yolo:[636, 638, 1018, 919], iou:0.848
frame 1, ground truth bb:[866, 581, 1170, 818], yolo:[636, 638

In [8]:
# from video_utils_virat import VideoUtils
# vUtils = VideoUtils() 
# vUtils.ChangeVideoFrameRate('video_20190906150731_3.avi','test2.avi', 10)

In [9]:
# from video_utils_virat import YoloUtils
# yUtils = YoloUtils()
# yolo_weight_file = "../../yolo/darknet/yolov3.weights"
# yolo_cfg_file = "../../yolo/darknet/cfg/yolov3.cfg"
# yolo_names_file = "../../yolo/darknet/data/coco.names"
# yUtils.YoloTheWebcam(yolo_weight_file, yolo_cfg_file, yolo_names_file)

In [10]:
# import os
# import pyautogui

# # this changes the working directory folder to where this script is in
# # os.chdir(os.path.dirname(os.path.abspath(__file__)))
# screenshot = pyautogui.screenshot()
# print(screenshot)

In [11]:
# import gi
# gi.require_version("Wnck", "3.0")
# from gi.repository import Wnck

# scr = Wnck.Screen.get_default()
# scr.force_update()
# print(scr.get_active_window().get_name())

In [12]:
# from video_utils_virat import YoloUtils
# yUtils = YoloUtils()
# yolo_weight_file = "../../yolo/darknet/yolov3.weights"
# yolo_cfg_file = "../../yolo/darknet/cfg/yolov3.cfg"
# yolo_names_file = "../../yolo/darknet/data/coco.names"
# yUtils.YoloOnScreen(yolo_weight_file, yolo_cfg_file, yolo_names_file, 0,0,1000,1000)
